In [22]:
import pandas as pd
from ast import literal_eval
from gensim.parsing import preprocessing
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models.keyedvectors import KeyedVectors
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from sklearn.model_selection import KFold

In [11]:
# Using GloVe as the word vector 
word_vec_link = 'http://nlp.standford.edu/data/glove.6B.zip' 
# Downloading the word vector
!wget http://nlp.stanford.edu/data/glove.6B.zip 
# Unzipping the word vector
!unzip glove*.zip

--2022-12-09 01:12:36--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-12-09 01:12:36--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-12-09 01:12:36--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [23]:
df = pd.read_csv('/content/drive/Shareddrives/Data-240 Project/hatexplain/hatexplain_final.csv')

In [24]:
df

,post_id,post_tokens,label_1,label_2,label_3,normal,offensive,hatespeech,vote
0,1179055004553900032_twitter,"['i', 'dont', 'think', 'im', 'getting', 'my', ...",normal,normal,normal,1,0,0,normal
1,1179063826874032128_twitter,"['we', 'cannot', 'continue', 'calling', 'ourse...",normal,normal,normal,1,0,0,normal
2,1178793830532956161_twitter,"['nawt', 'yall', 'niggers', 'ignoring', 'me']",normal,normal,hatespeech,1,0,1,normal
3,1179088797964763136_twitter,"['<user>', 'i', 'am', 'bit', 'confused', 'coz'...",hatespeech,offensive,hatespeech,0,1,1,hatespeech
4,1179085312976445440_twitter,"['this', 'bitch', 'in', 'whataburger', 'eating...",hatespeech,hatespeech,offensive,0,1,1,hatespeech
...,...,...,...,...,...,...,...,...,...
20143,9989999_gab,"['if', 'ur', 'still', 'on', 'twitter', 'tell',...",offensive,offensive,normal,1,1,0,offensive
20144,9990225_gab,"['when', 'i', 'first', 'got', 'on', 'here', 'a...",offensive,offensive,normal,1,1,0,offensive
20145,9991681_gab,"['was', 'macht', 'der', 'moslem', 'wenn', 'der...",offensive,normal,normal,1,1,0,normal
20146,9992513_gab,"['it', 'is', 'awful', 'look', 'at', 'world', '...",hatespeech,hatespeech,offensive,0,1,1,hatespeech


In [25]:
glove2word2vec(glove_input_file="glove.6B.200d.txt", word2vec_output_file="gensim_glove_vectors.txt")

(400000, 200)

In [26]:
df['tokens'] = None

for i,text in enumerate(df['post_tokens']):
    df['tokens'][i] = preprocessing.preprocess_string(text)
df

<ipython-input-26-59bbd658ae54>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tokens'][i] = preprocessing.preprocess_string(text)


,post_id,post_tokens,label_1,label_2,label_3,normal,offensive,hatespeech,vote,tokens
0,1179055004553900032_twitter,"['i', 'dont', 'think', 'im', 'getting', 'my', ...",normal,normal,normal,1,0,0,normal,"[dont, think, get, babi, white, white, nike, t..."
1,1179063826874032128_twitter,"['we', 'cannot', 'continue', 'calling', 'ourse...",normal,normal,normal,1,0,0,normal,"[continu, call, feminist, right, womxn, arent,..."
2,1178793830532956161_twitter,"['nawt', 'yall', 'niggers', 'ignoring', 'me']",normal,normal,hatespeech,1,0,1,normal,"[nawt, yall, nigger, ignor]"
3,1179088797964763136_twitter,"['<user>', 'i', 'am', 'bit', 'confused', 'coz'...",hatespeech,offensive,hatespeech,0,1,1,hatespeech,"[bit, confus, coz, chines, ppl, access, twitte..."
4,1179085312976445440_twitter,"['this', 'bitch', 'in', 'whataburger', 'eating...",hatespeech,hatespeech,offensive,0,1,1,hatespeech,"[bitch, whataburg, eat, burger, bun, hold, pal..."
...,...,...,...,...,...,...,...,...,...,...
20143,9989999_gab,"['if', 'ur', 'still', 'on', 'twitter', 'tell',...",offensive,offensive,normal,1,1,0,offensive,"[twitter, tell, carlton, said, alcohol, drug, ..."
20144,9990225_gab,"['when', 'i', 'first', 'got', 'on', 'here', 'a...",offensive,offensive,normal,1,1,0,offensive,"[got, said, hate, trump, consid, troll, peopl,..."
20145,9991681_gab,"['was', 'macht', 'der', 'moslem', 'wenn', 'der...",offensive,normal,normal,1,1,0,normal,"[macht, der, moslem, wenn, der, zion, gegen, s..."
20146,9992513_gab,"['it', 'is', 'awful', 'look', 'at', 'world', '...",hatespeech,hatespeech,offensive,0,1,1,hatespeech,"[aw, look, world, demograph, asian, fuck, betr..."


In [27]:
df['tokens_padded'] = None

for i, token_list in enumerate(df.tokens):
    temp = token_list.copy()
    if len(token_list) > 199:
        df['tokens_padded'][i] = token_list[:200]
    else:
        for _ in range(200-len(token_list)):
            temp.append('')
        df['tokens_padded'][i] = temp
df

<ipython-input-27-9d581a82796d>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tokens_padded'][i] = temp


,post_id,post_tokens,label_1,label_2,label_3,normal,offensive,hatespeech,vote,tokens,tokens_padded
0,1179055004553900032_twitter,"['i', 'dont', 'think', 'im', 'getting', 'my', ...",normal,normal,normal,1,0,0,normal,"[dont, think, get, babi, white, white, nike, t...","[dont, think, get, babi, white, white, nike, t..."
1,1179063826874032128_twitter,"['we', 'cannot', 'continue', 'calling', 'ourse...",normal,normal,normal,1,0,0,normal,"[continu, call, feminist, right, womxn, arent,...","[continu, call, feminist, right, womxn, arent,..."
2,1178793830532956161_twitter,"['nawt', 'yall', 'niggers', 'ignoring', 'me']",normal,normal,hatespeech,1,0,1,normal,"[nawt, yall, nigger, ignor]","[nawt, yall, nigger, ignor, , , , , , , , , , ..."
3,1179088797964763136_twitter,"['<user>', 'i', 'am', 'bit', 'confused', 'coz'...",hatespeech,offensive,hatespeech,0,1,1,hatespeech,"[bit, confus, coz, chines, ppl, access, twitte...","[bit, confus, coz, chines, ppl, access, twitte..."
4,1179085312976445440_twitter,"['this', 'bitch', 'in', 'whataburger', 'eating...",hatespeech,hatespeech,offensive,0,1,1,hatespeech,"[bitch, whataburg, eat, burger, bun, hold, pal...","[bitch, whataburg, eat, burger, bun, hold, pal..."
...,...,...,...,...,...,...,...,...,...,...,...
20143,9989999_gab,"['if', 'ur', 'still', 'on', 'twitter', 'tell',...",offensive,offensive,normal,1,1,0,offensive,"[twitter, tell, carlton, said, alcohol, drug, ...","[twitter, tell, carlton, said, alcohol, drug, ..."
20144,9990225_gab,"['when', 'i', 'first', 'got', 'on', 'here', 'a...",offensive,offensive,normal,1,1,0,offensive,"[got, said, hate, trump, consid, troll, peopl,...","[got, said, hate, trump, consid, troll, peopl,..."
20145,9991681_gab,"['was', 'macht', 'der', 'moslem', 'wenn', 'der...",offensive,normal,normal,1,1,0,normal,"[macht, der, moslem, wenn, der, zion, gegen, s...","[macht, der, moslem, wenn, der, zion, gegen, s..."
20146,9992513_gab,"['it', 'is', 'awful', 'look', 'at', 'world', '...",hatespeech,hatespeech,offensive,0,1,1,hatespeech,"[aw, look, world, demograph, asian, fuck, betr...","[aw, look, world, demograph, asian, fuck, betr..."


In [28]:
df['bigrams'] = None

for i,tokens in enumerate(df.tokens):
    df['bigrams'][i] = [tokens[i] +" "+ tokens[i+1] for i in range(len(tokens)-1)]
df

<ipython-input-28-3098876b8180>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['bigrams'][i] = [tokens[i] +" "+ tokens[i+1] for i in range(len(tokens)-1)]


,post_id,post_tokens,label_1,label_2,label_3,normal,offensive,hatespeech,vote,tokens,tokens_padded,bigrams
0,1179055004553900032_twitter,"['i', 'dont', 'think', 'im', 'getting', 'my', ...",normal,normal,normal,1,0,0,normal,"[dont, think, get, babi, white, white, nike, t...","[dont, think, get, babi, white, white, nike, t...","[dont think, think get, get babi, babi white, ..."
1,1179063826874032128_twitter,"['we', 'cannot', 'continue', 'calling', 'ourse...",normal,normal,normal,1,0,0,normal,"[continu, call, feminist, right, womxn, arent,...","[continu, call, feminist, right, womxn, arent,...","[continu call, call feminist, feminist right, ..."
2,1178793830532956161_twitter,"['nawt', 'yall', 'niggers', 'ignoring', 'me']",normal,normal,hatespeech,1,0,1,normal,"[nawt, yall, nigger, ignor]","[nawt, yall, nigger, ignor, , , , , , , , , , ...","[nawt yall, yall nigger, nigger ignor]"
3,1179088797964763136_twitter,"['<user>', 'i', 'am', 'bit', 'confused', 'coz'...",hatespeech,offensive,hatespeech,0,1,1,hatespeech,"[bit, confus, coz, chines, ppl, access, twitte...","[bit, confus, coz, chines, ppl, access, twitte...","[bit confus, confus coz, coz chines, chines pp..."
4,1179085312976445440_twitter,"['this', 'bitch', 'in', 'whataburger', 'eating...",hatespeech,hatespeech,offensive,0,1,1,hatespeech,"[bitch, whataburg, eat, burger, bun, hold, pal...","[bitch, whataburg, eat, burger, bun, hold, pal...","[bitch whataburg, whataburg eat, eat burger, b..."
...,...,...,...,...,...,...,...,...,...,...,...,...
20143,9989999_gab,"['if', 'ur', 'still', 'on', 'twitter', 'tell',...",offensive,offensive,normal,1,1,0,offensive,"[twitter, tell, carlton, said, alcohol, drug, ...","[twitter, tell, carlton, said, alcohol, drug, ...","[twitter tell, tell carlton, carlton said, sai..."
20144,9990225_gab,"['when', 'i', 'first', 'got', 'on', 'here', 'a...",offensive,offensive,normal,1,1,0,offensive,"[got, said, hate, trump, consid, troll, peopl,...","[got, said, hate, trump, consid, troll, peopl,...","[got said, said hate, hate trump, trump consid..."
20145,9991681_gab,"['was', 'macht', 'der', 'moslem', 'wenn', 'der...",offensive,normal,normal,1,1,0,normal,"[macht, der, moslem, wenn, der, zion, gegen, s...","[macht, der, moslem, wenn, der, zion, gegen, s...","[macht der, der moslem, moslem wenn, wenn der,..."
20146,9992513_gab,"['it', 'is', 'awful', 'look', 'at', 'world', '...",hatespeech,hatespeech,offensive,0,1,1,hatespeech,"[aw, look, world, demograph, asian, fuck, betr...","[aw, look, world, demograph, asian, fuck, betr...","[aw look, look world, world demograph, demogra..."


In [29]:
df['trigrams'] = None

for i,tokens in enumerate(df.tokens):
    df['trigrams'][i] = [tokens[i] +" "+ tokens[i+1] +" " + tokens[i+2] for i in range(len(tokens)-2)]
df

<ipython-input-29-a6222e6fcf5d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['trigrams'][i] = [tokens[i] +" "+ tokens[i+1] +" " + tokens[i+2] for i in range(len(tokens)-2)]


,post_id,post_tokens,label_1,label_2,label_3,normal,offensive,hatespeech,vote,tokens,tokens_padded,bigrams,trigrams
0,1179055004553900032_twitter,"['i', 'dont', 'think', 'im', 'getting', 'my', ...",normal,normal,normal,1,0,0,normal,"[dont, think, get, babi, white, white, nike, t...","[dont, think, get, babi, white, white, nike, t...","[dont think, think get, get babi, babi white, ...","[dont think get, think get babi, get babi whit..."
1,1179063826874032128_twitter,"['we', 'cannot', 'continue', 'calling', 'ourse...",normal,normal,normal,1,0,0,normal,"[continu, call, feminist, right, womxn, arent,...","[continu, call, feminist, right, womxn, arent,...","[continu call, call feminist, feminist right, ...","[continu call feminist, call feminist right, f..."
2,1178793830532956161_twitter,"['nawt', 'yall', 'niggers', 'ignoring', 'me']",normal,normal,hatespeech,1,0,1,normal,"[nawt, yall, nigger, ignor]","[nawt, yall, nigger, ignor, , , , , , , , , , ...","[nawt yall, yall nigger, nigger ignor]","[nawt yall nigger, yall nigger ignor]"
3,1179088797964763136_twitter,"['<user>', 'i', 'am', 'bit', 'confused', 'coz'...",hatespeech,offensive,hatespeech,0,1,1,hatespeech,"[bit, confus, coz, chines, ppl, access, twitte...","[bit, confus, coz, chines, ppl, access, twitte...","[bit confus, confus coz, coz chines, chines pp...","[bit confus coz, confus coz chines, coz chines..."
4,1179085312976445440_twitter,"['this', 'bitch', 'in', 'whataburger', 'eating...",hatespeech,hatespeech,offensive,0,1,1,hatespeech,"[bitch, whataburg, eat, burger, bun, hold, pal...","[bitch, whataburg, eat, burger, bun, hold, pal...","[bitch whataburg, whataburg eat, eat burger, b...","[bitch whataburg eat, whataburg eat burger, ea..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20143,9989999_gab,"['if', 'ur', 'still', 'on', 'twitter', 'tell',...",offensive,offensive,normal,1,1,0,offensive,"[twitter, tell, carlton, said, alcohol, drug, ...","[twitter, tell, carlton, said, alcohol, drug, ...","[twitter tell, tell carlton, carlton said, sai...","[twitter tell carlton, tell carlton said, carl..."
20144,9990225_gab,"['when', 'i', 'first', 'got', 'on', 'here', 'a...",offensive,offensive,normal,1,1,0,offensive,"[got, said, hate, trump, consid, troll, peopl,...","[got, said, hate, trump, consid, troll, peopl,...","[got said, said hate, hate trump, trump consid...","[got said hate, said hate trump, hate trump co..."
20145,9991681_gab,"['was', 'macht', 'der', 'moslem', 'wenn', 'der...",offensive,normal,normal,1,1,0,normal,"[macht, der, moslem, wenn, der, zion, gegen, s...","[macht, der, moslem, wenn, der, zion, gegen, s...","[macht der, der moslem, moslem wenn, wenn der,...","[macht der moslem, der moslem wenn, moslem wen..."
20146,9992513_gab,"['it', 'is', 'awful', 'look', 'at', 'world', '...",hatespeech,hatespeech,offensive,0,1,1,hatespeech,"[aw, look, world, demograph, asian, fuck, betr...","[aw, look, world, demograph, asian, fuck, betr...","[aw look, look world, world demograph, demogra...","[aw look world, look world demograph, world de..."


In [31]:
model_glove = KeyedVectors.load_word2vec_format("gensim_glove_vectors.txt", binary=False)

In [36]:
model_glove

In [30]:
df['word_index_glove'] = None

num_tokens_glove = len(model_glove)+2
embedding_dim_glove = len(model_glove[0])
embedding_matrix_glove = np.zeros((num_tokens_glove,embedding_dim_glove))

for i in range(2,num_tokens_glove):
    embedding_matrix_glove[i] = model_glove[i-2]

for i,token_list in enumerate(df['tokens']):
    output = []
    for token in token_list[:200]:
        try:
            output.append(model_glove.get_index(token))
        except:
            output.append(1)
    for _ in range(200-len(token_list)):
        output.append(1)
    df['word_index_glove'][i] = np.array(output)
df.head()

TypeError: ignored

In [18]:
X = tf.constant(list(df['word_index_glove']))
y = tf.convert_to_tensor(list(df['vote_ordinal']))

ValueError: ignored

In [10]:
embedding_layer = Embedding(
    num_tokens_glove,
    embedding_dim_glove,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix_glove),
    trainable=False,
)

int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.LSTM(200,go_backwards=True)(embedded_sequences)
#x = layers.Dropout(0.2)(x)
preds = layers.Dense(4, activation="softmax")(x)
model_lstm_glove = keras.Model(int_sequences_input, preds)
model_lstm_glove.summary()

NameError: ignored

In [ ]:
intermediate_layer_model = keras.Model(inputs=model_lstm_glove.input,
                                       outputs=model_lstm_glove.get_layer('lstm').output)
intermediate_output = intermediate_layer_model(X)

In [ ]:
model_lstm_glove.compile(loss=keras.losses.SparseCategoricalCrossentropy(),optimizer='rmsprop',metrics=['acc'])
model_lstm_glove.fit(X, y, batch_size=32, epochs= 10)

Epoch 1/10
630/630 [==============================] - 46s 71ms/step - loss: 1.0533 - acc: 0.5597
Epoch 2/10
630/630 [==============================] - 45s 71ms/step - loss: 0.9675 - acc: 0.6057
Epoch 3/10
630/630 [==============================] - 46s 73ms/step - loss: 0.9143 - acc: 0.6289
Epoch 4/10
630/630 [==============================] - 45s 72ms/step - loss: 0.8623 - acc: 0.6474
Epoch 5/10
630/630 [==============================] - 45s 71ms/step - loss: 0.7885 - acc: 0.6880
Epoch 6/10
630/630 [==============================] - 45s 71ms/step - loss: 0.6842 - acc: 0.7281
Epoch 7/10
630/630 [==============================] - 45s 71ms/step - loss: 0.5651 - acc: 0.7836
Epoch 8/10
630/630 [==============================] - 45s 71ms/step - loss: 0.4399 - acc: 0.8389
Epoch 9/10
630/630 [==============================] - 45s 71ms/step - loss: 0.3305 - acc: 0.8827
Epoch 10/10
630/630 [==============================] - 47s 75ms/step - loss: 0.2420 - acc: 0.9180


In [ ]:
df['lstm_feature_map'] = None
for i, array in enumerate(intermediate_output):
    df['lstm_feature_map'][i] = list(array.numpy())
df

C:\Users\geewh\AppData\Local\Temp\ipykernel_25288\608061594.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['lstm_feature_map'][i] = list(array.numpy())


,post_id,post_tokens,label_1,label_2,label_3,vote,vote_ordinal,tokens,tokens_padded,bigrams,trigrams,word_index_glove,lstm_feature_map
0,1179055004553900032_twitter,"['i', 'dont', 'think', 'im', 'getting', 'my', ...",normal,normal,normal,normal,1,"[dont, think, get, babi, white, white, nike, t...","[dont, think, get, babi, white, white, nike, t...","[dont think, think get, get babi, babi white, ...","[dont think get, think get babi, get babi whit...","[46768, 269, 169, 71222, 298, 298, 10625, 3255...","[0.13938053, -0.18590486, -0.060539033, 0.0901..."
1,1179063826874032128_twitter,"['we', 'cannot', 'continue', 'calling', 'ourse...",normal,normal,normal,normal,1,"[continu, call, feminist, right, womxn, arent,...","[continu, call, feminist, right, womxn, arent,...","[continu call, call feminist, feminist right, ...","[continu call feminist, call feminist right, f...","[1, 580, 11853, 248, 1, 138121, 1476, 15082, 2...","[0.04327799, -0.08438103, -0.060883567, 0.0025..."
2,1178793830532956161_twitter,"['nawt', 'yall', 'niggers', 'ignoring', 'me']",normal,normal,hatespeech,normal,1,"[nawt, yall, nigger, ignor]","[nawt, yall, nigger, ignor, , , , , , , , , , ...","[nawt yall, yall nigger, nigger ignor]","[nawt yall nigger, yall nigger ignor]","[1, 381085, 42443, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0.030954387, 0.040644534, -0.07671745, 0.0194..."
3,1179088797964763136_twitter,"['<user>', 'i', 'am', 'bit', 'confused', 'coz'...",hatespeech,offensive,hatespeech,hatespeech,3,"[bit, confus, coz, chines, ppl, access, twitte...","[bit, confus, coz, chines, ppl, access, twitte...","[bit confus, confus coz, coz chines, chines pp...","[bit confus coz, confus coz chines, coz chines...","[1594, 1, 120621, 156181, 65387, 1122, 10360, ...","[0.004926282, 0.0012388171, -0.0033861813, 0.0..."
4,1179085312976445440_twitter,"['this', 'bitch', 'in', 'whataburger', 'eating...",hatespeech,hatespeech,offensive,hatespeech,3,"[bitch, whataburg, eat, burger, bun, hold, pal...","[bitch, whataburg, eat, burger, bun, hold, pal...","[bitch whataburg, whataburg eat, eat burger, b...","[bitch whataburg eat, whataburg eat burger, ea...","[30619, 1, 3623, 12203, 28254, 802, 3214, 5281...","[0.12814277, -0.02799369, -0.022903804, -0.103..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20143,9989999_gab,"['if', 'ur', 'still', 'on', 'twitter', 'tell',...",offensive,offensive,normal,offensive,2,"[twitter, tell, carlton, said, alcohol, drug, ...","[twitter, tell, carlton, said, alcohol, drug, ...","[twitter tell, tell carlton, carlton said, sai...","[twitter tell carlton, tell carlton said, carl...","[10360, 1361, 11828, 16, 4027, 780, 21675, 1, ...","[-0.08081896, -0.25950265, 0.078297645, 0.0035..."
20144,9990225_gab,"['when', 'i', 'first', 'got', 'on', 'here', 'a...",offensive,offensive,normal,offensive,2,"[got, said, hate, trump, consid, troll, peopl,...","[got, said, hate, trump, consid, troll, peopl,...","[got said, said hate, hate trump, trump consid...","[got said hate, said hate trump, hate trump co...","[405, 16, 5281, 10468, 1, 38846, 1, 22696, 528...","[0.020776447, -0.17637987, 0.11507965, 0.00168..."
20145,9991681_gab,"['was', 'macht', 'der', 'moslem', 'wenn', 'der...",offensive,normal,normal,normal,1,"[macht, der, moslem, wenn, der, zion, gegen, s...","[macht, der, moslem, wenn, der, zion, gegen, s...","[macht der, der moslem, moslem wenn, wenn der,...","[macht der moslem, der moslem wenn, moslem wen...","[69557, 5027, 7449, 102626, 5027, 20971, 13442...","[0.030287618, -0.091516644, -0.115807004, -0.0..."
20146,9992513_gab,"['it', 'is', 'awful', 'look', 'at', 'world', '...",hatespeech,hatespeech,offensive,hatespeech,3,"[aw, look, world, demograph, asian, fuck, betr...","[aw, look, world, demograph, asian, fuck, betr...","[aw look, look world, world demograph, demogra...","[aw look world, look world demograph, world de...","[34849, 662, 85, 1, 967, 35690, 28379, 298, 1,...","[-0.01178721, -0.16269265, -0.07008049, 0.0428..."


In [ ]:
# https://www.davidsbatista.net/blog/2018/02/28/TfidfVectorizer/

def dummy_fun(doc):
    return doc

df['tfidf_word1gram'] = None

tfidf_vec = TfidfVectorizer(analyzer='word',tokenizer=dummy_fun,preprocessor=dummy_fun,token_pattern=None,ngram_range=(1,1))
tfidf_vec.fit(df.tokens)

for i,tokens in enumerate(df.tokens):
    tfidf_list = []
    for token in tokens:
        tfidf_list.append(tfidf_vec.vocabulary_.get(token))
    df['tfidf_word1gram'][i] = tfidf_list

C:\Users\geewh\AppData\Local\Temp\ipykernel_25288\361513316.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tfidf_word1gram'][i] = tfidf_list


In [ ]:
df['tfidf_word2gram'] = None

tfidf_vec = TfidfVectorizer(analyzer='word',tokenizer=dummy_fun,preprocessor=dummy_fun,token_pattern=None,ngram_range=(2,2))
#tfidf_vec = TfidfVectorizer(analyzer='word',ngram_range=(2,2))
tfidf_vec.fit(df.tokens)

for i,tokens in enumerate(df.bigrams):
    tfidf_list = []
    for token in tokens:
        tfidf_list.append(tfidf_vec.vocabulary_.get(token))
    df['tfidf_word2gram'][i] = tfidf_list
df

C:\Users\geewh\AppData\Local\Temp\ipykernel_25288\276389147.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tfidf_word2gram'][i] = tfidf_list


,post_id,post_tokens,label_1,label_2,label_3,vote,vote_ordinal,tokens,tokens_padded,bigrams,trigrams,word_index_glove,lstm_feature_map,tfidf_word1gram,tfidf_word2gram
0,1179055004553900032_twitter,"['i', 'dont', 'think', 'im', 'getting', 'my', ...",normal,normal,normal,normal,1,"[dont, think, get, babi, white, white, nike, t...","[dont, think, get, babi, white, white, nike, t...","[dont think, think get, get babi, babi white, ...","[dont think get, think get babi, get babi whit...","[46768, 269, 169, 71222, 298, 298, 10625, 3255...","[0.13938053, -0.18590486, -0.060539033, 0.0901...","[4834, 17394, 6750, 1188, 19043, 19043, 11943,...","[36687, 139476, 52026, 9263, 153473, 153092, 9..."
1,1179063826874032128_twitter,"['we', 'cannot', 'continue', 'calling', 'ourse...",normal,normal,normal,normal,1,"[continu, call, feminist, right, womxn, arent,...","[continu, call, feminist, right, womxn, arent,...","[continu call, call feminist, feminist right, ...","[continu call feminist, call feminist right, f...","[1, 580, 11853, 248, 1, 138121, 1476, 15082, 2...","[0.04327799, -0.08438103, -0.060883567, 0.0025...","[3571, 2512, 5937, 14569, 19264, 832, 168, 194...","[27018, 18616, 45642, 119516, 155555, 6181, 15..."
2,1178793830532956161_twitter,"['nawt', 'yall', 'niggers', 'ignoring', 'me']",normal,normal,hatespeech,normal,1,"[nawt, yall, nigger, ignor]","[nawt, yall, nigger, ignor, , , , , , , , , , ...","[nawt yall, yall nigger, nigger ignor]","[nawt yall nigger, yall nigger ignor]","[1, 381085, 42443, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0.030954387, 0.040644534, -0.07671745, 0.0194...","[11667, 19441, 11893, 8233]","[94118, 157521, 97062]"
3,1179088797964763136_twitter,"['<user>', 'i', 'am', 'bit', 'confused', 'coz'...",hatespeech,offensive,hatespeech,hatespeech,3,"[bit, confus, coz, chines, ppl, access, twitte...","[bit, confus, coz, chines, ppl, access, twitte...","[bit confus, confus coz, coz chines, chines pp...","[bit confus coz, confus coz chines, coz chines...","[1594, 1, 120621, 156181, 65387, 1122, 10360, ...","[0.004926282, 0.0012388171, -0.0033861813, 0.0...","[1757, 3491, 3743, 2964, 13434, 97, 17994, 174...","[12858, 26378, 29004, 22211, 108082, 598, 1454..."
4,1179085312976445440_twitter,"['this', 'bitch', 'in', 'whataburger', 'eating...",hatespeech,hatespeech,offensive,hatespeech,3,"[bitch, whataburg, eat, burger, bun, hold, pal...","[bitch, whataburg, eat, burger, bun, hold, pal...","[bitch whataburg, whataburg eat, eat burger, b...","[bitch whataburg eat, whataburg eat burger, ea...","[30619, 1, 3623, 12203, 28254, 802, 3214, 5281...","[0.12814277, -0.02799369, -0.022903804, -0.103...","[1758, 19000, 5128, 2393, 2373, 7844, 12652, 7...","[13509, 152337, 38652, 18022, 17904, 62123, 10..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20143,9989999_gab,"['if', 'ur', 'still', 'on', 'twitter', 'tell',...",offensive,offensive,normal,offensive,2,"[twitter, tell, carlton, said, alcohol, drug, ...","[twitter, tell, carlton, said, alcohol, drug, ...","[twitter tell, tell carlton, carlton said, sai...","[twitter tell carlton, tell carlton said, carl...","[10360, 1361, 11828, 16, 4027, 780, 21675, 1, ...","[-0.08081896, -0.25950265, 0.078297645, 0.0035...","[17994, 17207, 2640, 14893, 403, 4982, 164, 17...","[145398, 137813, 19819, 121393, 3077, 37458, 1..."
20144,9990225_gab,"['when', 'i', 'first', 'got', 'on', 'here', 'a...",offensive,offensive,normal,offensive,2,"[got, said, hate, trump, consid, troll, peopl,...","[got, said, hate, trump, consid, troll, peopl,...","[got said, said hate, hate trump, trump consid...","[got said hate, said hate trump, hate trump co...","[405, 16, 5281, 10468, 1, 38846, 1, 22696, 528...","[0.020776447, -0.17637987, 0.11507965, 0.00168...","[6997, 14893, 7509, 17862, 3533, 17828, 12902,...","[55464, 121497, 59401, 143889, 26734, 143516, ..."
20145,9991681_gab,"['was', 'macht', 'der', 'moslem', 'wenn', 'der...",offensive,normal,normal,normal,1,"[macht, der, moslem, wenn, der, zion, g

In [ ]:
df['tfidf_word3gram'] = None

tfidf_vec = TfidfVectorizer(analyzer='word',tokenizer=dummy_fun,preprocessor=dummy_fun,token_pattern=None,ngram_range=(3,3))
#tfidf_vec = TfidfVectorizer(analyzer='word',ngram_range=(2,2))
tfidf_vec.fit(df.tokens)

for i,tokens in enumerate(df.trigrams):
    tfidf_list = []
    for token in tokens:
        tfidf_list.append(tfidf_vec.vocabulary_.get(token))
    df['tfidf_word3gram'][i] = tfidf_list

C:\Users\geewh\AppData\Local\Temp\ipykernel_25288\1671843974.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tfidf_word3gram'][i] = tfidf_list


In [ ]:
df['w2v_cbow'] = None

model_w2v = Word2Vec(sentences=df.tokens)
word_vectors_w2v = model_w2v.wv
num_tokens_w2v = len(word_vectors_w2v) + 2

# capturing W2V index of tokens
for i,token_list in enumerate(df.tokens):
    output = []
    for token in token_list[:200]:
        try:
            output.append(word_vectors_w2v.get_index(token))
        except:
            output.append(1)
    for _ in range(200-len(token_list)):
        output.append(1)
    df['w2v_cbow'][i] = np.array(output)

C:\Users\geewh\AppData\Local\Temp\ipykernel_25288\2164468804.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['w2v_cbow'][i] = np.array(output)


In [ ]:
for i, thing in enumerate(df.word_index_glove):
    df['word_index_glove'][i] = list(thing)

C:\Users\geewh\AppData\Local\Temp\ipykernel_25288\3701986464.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['word_index_glove'][i] = list(thing)


In [ ]:
for i, thing in enumerate(df.w2v_cbow):
    df['w2v_cbow'][i] = list(thing)

C:\Users\geewh\AppData\Local\Temp\ipykernel_25288\3385890829.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['w2v_cbow'][i] = list(thing)


In [ ]:
df[['vote_ordinal','word_index_glove','lstm_feature_map','tfidf_word1gram','tfidf_word2gram','tfidf_word3gram','w2v_cbow']].to_csv('hatexplain_feature_extractions_v2.csv')

In [ ]:
df2 = pd.read_csv('hatexplain_feature_extractions_v2.csv', 
                  converters={'word_index_glove': literal_eval, 
                              'lstm_feature_map': literal_eval,
                              'tfidf_word1gram': literal_eval,
                              'tfidf_word2gram': literal_eval,
                              'tfidf_word3gram': literal_eval,
                              'w2v_cbow': literal_eval})

In [ ]:
df2

,Unnamed: 0,vote_ordinal,word_index_glove,lstm_feature_map,tfidf_word1gram,tfidf_word2gram,tfidf_word3gram,w2v_cbow
0,0,1,"[46768, 269, 169, 71222, 298, 298, 10625, 3255...","[0.13938053, -0.18590486, -0.060539033, 0.0901...","[4834, 17394, 6750, 1188, 19043, 19043, 11943,...","[36687, 139476, 52026, 9263, 153473, 153092, 9...","[39463, 155726, 56397, 10094, 173291, 172093]","[161, 21, 70, 155, 0, 0, 4001, 988, 1, 1, 1, 1..."
1,1,1,"[1, 580, 11853, 248, 1, 138121, 1476, 15082, 2...","[0.04327799, -0.08438103, -0.060883567, 0.0025...","[3571, 2512, 5937, 14569, 19264, 832, 168, 194...","[27018, 18616, 45642, 119516, 155555, 6181, 15...","[29372, 20476, 48977, 134079, 176031, 6848, 16...","[565, 40, 326, 23, 1, 2026, 1264, 118, 167, 1,..."
2,2,1,"[1, 381085, 42443, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0.030954387, 0.040644534, -0.07671745, 0.0194...","[11667, 19441, 11893, 8233]","[94118, 157521, 97062]","[105012, 178166]","[1, 672, 1, 279, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,..."
3,3,3,"[1594, 1, 120621, 156181, 65387, 1122, 10360, ...","[0.004926282, 0.0012388171, -0.0033861813, 0.0...","[1757, 3491, 3743, 2964, 13434, 97, 17994, 174...","[12858, 26378, 29004, 22211, 108082, 598, 1454...","[13838, 28731, 31520, 24268, 121193, 620, 1623...","[454, 987, 2862, 640, 221, 1793, 81, 1, 569, 5..."
4,4,3,"[30619, 1, 3623, 12203, 28254, 802, 3214, 5281...","[0.12814277, -0.02799369, -0.022903804, -0.103...","[1758, 19000, 5128, 2393, 2373, 7844, 12652, 7...","[13509, 152337, 38652, 18022, 17904, 62123, 10...","[14667, 169988, 41588, 19867, 19736, 67803, 11...","[10, 1, 313, 3335, 1, 458, 1, 14, 0, 10, 1, 1,..."
...,...,...,...,...,...,...,...,...
20143,20143,2,"[10360, 1361, 11828, 16, 4027, 780, 21675, 1, ...","[-0.08081896, -0.25950265, 0.078297645, 0.0035...","[17994, 17207, 2640, 14893, 403, 4982, 164, 17...","[145398, 137813, 19819, 121393, 3077, 37458, 1...","[162339, 153834, 21830, 136104, 3273, 40289, 1...","[81, 79, 1, 56, 1763, 457, 1861, 1934, 292, 47..."
20144,20144,2,"[405, 16, 5281, 10468, 1, 38846, 1, 22696, 528...","[0.020776447, -0.17637987, 0.11507965, 0.00168...","[6997, 14893, 7509, 17862, 3533, 17828, 12902,...","[55464, 121497, 59401, 143889, 26734, 143516, ...","[60568, 136229, 64905, 160529, 29083, 160148, ...","[41, 56, 14, 29, 396, 622, 4, 1699, 14, 1, 622..."
20145,20145,1,"[69557, 5027, 7449, 102626, 5027, 20971, 13442...","[0.030287618, -0.091516644, -0.115807004, -0.0...","[10311, 4359, 11291, 18950, 4359, 19629, 6680,...","[83848, 33894, 90844, 151973, 33896, 158813, 5...","[93270, 36609, 101069, 169588, 36611, 179677, ...","[1, 2887, 35, 1, 2887, 3239, 1, 1, 1, 1, 1, 1,..."
20146,20146,3,"[34849, 662, 85, 1, 967, 35690, 28379, 298, 1,...","[-0.01178721, -0.16269265, -0.07008049, 0.0428...","[1132, 10144, 19311, 4288, 932, 6419, 1629, 19...","[8861, 82483, 156365, 33511, 6904, 48894, 1198...","[9682, 91762, 176956, 36243, 7573, 52363, 1292...","[1715, 25, 63, 1608, 43, 3, 1, 0, 357, 82, 1, ..."


In [ ]:
for i,token_list in enumerate(df.tfidf_word1gram):
    if len(token_list) > 199:
        df['tfidf_word1gram'][i] = token_list[:200]
    else:
        for _ in range(200-len(token_list)):
            token_list.append(0)
        df['tfidf_word1gram'][i] = token_list
for i,token_list in enumerate(df.tfidf_word2gram):
    if len(token_list) > 199:
        df['tfidf_word2gram'][i] = token_list[:200]
    else:
        for _ in range(200-len(token_list)):
            token_list.append(0)
        df['tfidf_word2gram'][i] = token_list
for i,token_list in enumerate(df.tfidf_word3gram):
    if len(token_list) > 199:
        df['tfidf_word3gram'][i] = token_list[:200]
    else:
        for _ in range(200-len(token_list)):
            token_list.append(0)
        df['tfidf_word3gram'][i] = token_list
df.head()

C:\Users\geewh\AppData\Local\Temp\ipykernel_25288\3397904912.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tfidf_word1gram'][i] = token_list[:200]
C:\Users\geewh\AppData\Local\Temp\ipykernel_25288\3397904912.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tfidf_word2gram'][i] = token_list
C:\Users\geewh\AppData\Local\Temp\ipykernel_25288\3397904912.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tfidf_word3g

,post_id,post_tokens,label_1,label_2,label_3,vote,vote_ordinal,tokens,tokens_padded,bigrams,trigrams,word_index_glove,lstm_feature_map,tfidf_word1gram,tfidf_word2gram,tfidf_word3gram,w2v_cbow,pos_tag
0,1179055004553900032_twitter,"['i', 'dont', 'think', 'im', 'getting', 'my', ...",normal,normal,normal,normal,1,"[dont, think, get, babi, white, white, nike, t...","[dont, think, get, babi, white, white, nike, t...","[dont think, think get, get babi, babi white, ...","[dont think get, think get babi, get babi whit...","[46768, 269, 169, 71222, 298, 298, 10625, 3255...","[0.13938053, -0.18590486, -0.060539033, 0.0901...","[4834, 17394, 6750, 1188, 19043, 19043, 11943,...","[36687, 139476, 52026, 9263, 153473, 153092, 9...","[39463, 155726, 56397, 10094, 173291, 172093, ...","[161, 21, 70, 155, 0, 0, 4001, 988, 1, 1, 1, 1...","[NN, VBP, VB, JJ, JJ, JJ, NN, NN, NNP, NNP, NN..."
1,1179063826874032128_twitter,"['we', 'cannot', 'continue', 'calling', 'ourse...",normal,normal,normal,normal,1,"[continu, call, feminist, right, womxn, arent,...","[continu, call, feminist, right, womxn, arent,...","[continu call, call feminist, feminist right, ...","[continu call feminist, call feminist right, f...","[1, 580, 11853, 248, 1, 138121, 1476, 15082, 2...","[0.04327799, -0.08438103, -0.060883567, 0.0025...","[3571, 2512, 5937, 14569, 19264, 832, 168, 194...","[27018, 18616, 45642, 119516, 155555, 6181, 15...","[29372, 20476, 48977, 134079, 176031, 6848, 16...","[565, 40, 326, 23, 1, 2026, 1264, 118, 167, 1,...","[NN, NN, NN, NN, NN, NN, NN, VBP, JJ, IN, JJ, ..."
2,1178793830532956161_twitter,"['nawt', 'yall', 'niggers', 'ignoring', 'me']",normal,normal,hatespeech,normal,1,"[nawt, yall, nigger, ignor]","[nawt, yall, nigger, ignor, , , , , , , , , , ...","[nawt yall, yall nigger, nigger ignor]","[nawt yall nigger, yall nigger ignor]","[1, 381085, 42443, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0.030954387, 0.040644534, -0.07671745, 0.0194...","[11667, 19441, 11893, 8233, 0, 0, 0, 0, 0, 0, ...","[94118, 157521, 97062, 0, 0, 0, 0, 0, 0, 0, 0,...","[105012, 178166, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[1, 672, 1, 279, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[RB, RB, JJ, JJ, NN, NNP, NNP, NNP, NNP, NNP, ..."
3,1179088797964763136_twitter,"['<user>', 'i', 'am', 'bit', 'confused', 'coz'...",hatespeech,offensive,hatespeech,hatespeech,3,"[bit, confus, coz, chines, ppl, access, twitte...","[bit, confus, coz, chines, ppl, access, twitte...","[bit confus, confus coz, coz chines, chines pp...","[bit confus coz, confus coz chines, coz chines...","[1594, 1, 120621, 156181, 65387, 1122, 10360, ...","[0.004926282, 0.0012388171, -0.0033861813, 0.0...","[1757, 3491, 3743, 2964, 13434, 97, 17994, 174...","[12858, 26378, 29004, 22211, 108082, 598, 1454...","[13838, 28731, 31520, 24268, 121193, 620, 1623...","[454, 987, 2862, 640, 221, 1793, 81, 1, 569, 5...","[NN, JJ, NN, NNS, JJ, NN, NN, NN, VBG, JJ, VBP..."
4,1179085312976445440_twitter,"['this', 'bitch', 'in', 'whataburger', 'eating...",hatespeech,hatespeech,offensive,hatespeech,3,"[bitch, whataburg, eat, burger, bun, hold, pal...","[bitch, whataburg, eat, burger, bun, hold, pal...","[bitch whataburg, whataburg eat, eat burger, b...","[bitch whataburg eat, whataburg eat burger, ea...","[30619, 1, 3623, 12203, 28254, 802, 3214, 5281...","[0.12814277, -0.02799369, -0.022903804, -0.103...","[1758, 19000, 5128, 2393, 2373, 7844, 12652, 7...","[13509, 152337, 38652, 18022, 17904, 62123, 10...","[14667, 169988, 41588, 19867, 19736, 67803, 11...","[10, 1, 313, 3335, 1, 458, 1, 14, 0, 10, 1, 1,...","[NN, NN, NN, NN, NN, VBP, JJ, JJ, JJ, NN, NN, ..."


In [ ]:
df[['vote_ordinal','word_index_glove','lstm_feature_map','tfidf_word1gram','tfidf_word2gram','tfidf_word3gram','w2v_cbow']].to_csv('hatexplain_feature_extractions_v3.csv')